In [1]:
from findatapy.market import Market, MarketDataRequest, MarketDataGenerator

In [2]:
market = Market(market_data_generator=MarketDataGenerator())

# first we can do it by defining all the vendor fields, tickers etc. so we bypass the configuration file
md_request = MarketDataRequest( start_date='14 Jun 2016',
                                finish_date='20 Jun 2016',
                                fields=['bid', 'ask'], 
                                vendor_fields=['bid', 'ask'],
                                freq='tick', 
                                data_source='dukascopy',
                                tickers=['EURUSD'],
                                vendor_tickers=['EURUSD'])

df = market.fetch_market(md_request)
print(df.tail(n=10))

Couldn't push MarketDataRequest_858__abstract_curve_key-None__base_depos_currencies-EUR_GBP_AUD_NZD_USD_CAD_CHF_NOK_SEK_JPY__base_depos_tenor-ON_TN_SN_1W_2W_3W_1M_... to Redis: Error 61 connecting to 127.0.0.1:6379. Connection refused.


                                  EURUSD.bid  EURUSD.ask
Date                                                    
2016-06-19 23:59:48.959000+00:00     1.13471     1.13476
2016-06-19 23:59:51.179000+00:00     1.13471     1.13476
2016-06-19 23:59:53.703000+00:00     1.13471     1.13476
2016-06-19 23:59:54.226000+00:00     1.13470     1.13476
2016-06-19 23:59:54.813000+00:00     1.13471     1.13476
2016-06-19 23:59:55.321000+00:00     1.13470     1.13476
2016-06-19 23:59:55.834000+00:00     1.13471     1.13476
2016-06-19 23:59:58.713000+00:00     1.13470     1.13476
2016-06-19 23:59:59.228000+00:00     1.13470     1.13475
2016-06-19 23:59:59.746000+00:00     1.13469     1.13474


In [3]:
df['EURUSD.mid'] = (df['EURUSD.ask'] + df['EURUSD.bid']) / 2.0

In [4]:
def get_tick_data(start_date,finish_date):
    market = Market(market_data_generator=MarketDataGenerator())

    # first we can do it by defining all the vendor fields, tickers etc. so we bypass the configuration file
    md_request = MarketDataRequest( start_date=start_date,
                                finish_date=finish_date,
                                fields=['bid', 'ask'], 
                                vendor_fields=['bid', 'ask'],
                                freq='tick', 
                                data_source='dukascopy',
                                tickers=['EURUSD'],
                                vendor_tickers=['EURUSD'])
    df = market.fetch_market(md_request)
    df['EURUSD.mid'] = (df['EURUSD.ask'] + df['EURUSD.bid']) / 2.0
    return df

In [7]:
tick_data = get_tick_data('14 Jun 2016','20 Jun 2016')

Couldn't push MarketDataRequest_858__abstract_curve_key-None__base_depos_currencies-EUR_GBP_AUD_NZD_USD_CAD_CHF_NOK_SEK_JPY__base_depos_tenor-ON_TN_SN_1W_2W_3W_1M_... to Redis: Error 61 connecting to 127.0.0.1:6379. Connection refused.


In [12]:
import pandas as pd

def tick_to_ohlc(tick_df: pd.DataFrame, timeframe: str) -> pd.DataFrame:
    '''
    Convert tick data to OHLC data.
    1 min = 1T
    1 Hr = 1H
    1 Day = 1D
    '''
    # Assuming the DataFrame is indexed by timestamp and has a 'Mid' column
    # for the mid prices. Adapt as necessary.

    # Resample the tick data to OHLC data using the specified timeframe
    ohlc_df = tick_df['EURUSD.mid'].resample(timeframe).ohlc()
    
    # Drop rows where all values are NaN (which may happen in less active trading periods)
    ohlc_df.dropna(how='all', inplace=True)
    
    return ohlc_df

In [13]:
ohlc_df = tick_to_ohlc(tick_data, '1T')  # '1T' denotes 1-minute bars

print(ohlc_df)

                               open      high       low     close
Date                                                             
2016-06-14 00:00:00+00:00  1.128795  1.128795  1.128620  1.128725
2016-06-14 00:01:00+00:00  1.128715  1.128715  1.128635  1.128635
2016-06-14 00:02:00+00:00  1.128635  1.128755  1.128555  1.128735
2016-06-14 00:03:00+00:00  1.128735  1.128775  1.128645  1.128645
2016-06-14 00:04:00+00:00  1.128640  1.128695  1.128630  1.128670
...                             ...       ...       ...       ...
2016-06-19 23:55:00+00:00  1.135555  1.135595  1.134920  1.134925
2016-06-19 23:56:00+00:00  1.134935  1.135015  1.134210  1.134280
2016-06-19 23:57:00+00:00  1.134285  1.134740  1.134230  1.134710
2016-06-19 23:58:00+00:00  1.134705  1.134705  1.134550  1.134610
2016-06-19 23:59:00+00:00  1.134600  1.134815  1.134565  1.134715

[5757 rows x 4 columns]


In [15]:
def get_candlestick_data(start_date,finish_date,timeframe: str):
    '''get candlestick data from start_date to finish_date with timeframe
    date format: 'dd mmm yyyy' like '14 Jun 2016'
    timeframe format:
    1 min = 1T
    1 Hr = 1H
    1 Day = 1D
    '''
    tick_data = get_tick_data(start_date,finish_date)
    ohlc_df = tick_to_ohlc(tick_data, timeframe)
    return ohlc_df

In [16]:
get_candlestick_data('14 Jun 2016','20 Jun 2016','15T')

Couldn't push MarketDataRequest_858__abstract_curve_key-None__base_depos_currencies-EUR_GBP_AUD_NZD_USD_CAD_CHF_NOK_SEK_JPY__base_depos_tenor-ON_TN_SN_1W_2W_3W_1M_... to Redis: Error 61 connecting to 127.0.0.1:6379. Connection refused.


,open,high,low,close
Date,,,,
2016-06-14 00:00:00+00:00,1.128795,1.129015,1.128555,1.128895
2016-06-14 00:15:00+00:00,1.128880,1.129185,1.128825,1.128935
2016-06-14 00:30:00+00:00,1.128940,1.129115,1.128840,1.129035
2016-06-14 00:45:00+00:00,1.129030,1.129545,1.128930,1.129535
2016-06-14 01:00:00+00:00,1.129535,1.129720,1.129395,1.129625
...,...,...,...,...
2016-06-19 22:45:00+00:00,1.131670,1.132380,1.131440,1.132305
2016-06-19 23:00:00+00:00,1.132295,1.133530,1.132295,1.133420
2016-06-19 23:15:00+00:00,1.133425,1.133570,1.132930,1.133530


# Tasks
test function for stability while procuring larger datasets 
ignore the market close times 